In [1]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [2]:
from src.inference import get_feature_store

In [3]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from ),
    end_time=(fetch_data_to),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-05 02:08:09,918 INFO: Initializing external client
2025-03-05 02:08:09,922 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 02:08:12,355 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215683
Fetching data from 2025-02-04 07:08:09.822262+00:00 to 2025-03-05 06:08:09.822262+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (8.62s) 


In [4]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-04 08:00:00+00:00,2,0
1,2025-02-04 09:00:00+00:00,2,0
2,2025-02-04 10:00:00+00:00,2,0
3,2025-02-04 11:00:00+00:00,2,0
4,2025-02-04 12:00:00+00:00,2,0
...,...,...,...
175112,2025-03-05 02:00:00+00:00,263,5
175113,2025-03-05 03:00:00+00:00,263,3
175114,2025-03-05 04:00:00+00:00,263,4
175115,2025-03-05 05:00:00+00:00,263,19


In [5]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175117 entries, 0 to 175116
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         175117 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  175117 non-null  int32                  
 2   rides               175117 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 2.7 MB


In [6]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [7]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175117 entries, 0 to 175116
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         175117 non-null  datetime64[us]
 1   pickup_location_id  175117 non-null  int32         
 2   rides               175117 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 2.7 MB


In [8]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

Skipping location_id 176: Not enough data to create even one window.


In [9]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,32,48,145,155,6,6,14,86,14,106,...,49,133,55,137,148,132,108,39,263,2025-02-20 10:00:00
1,2,0,0,0,0,0,0,3,0,0,...,2,0,0,0,18,0,0,3,255,2025-02-17 18:00:00
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,0,210,2025-02-20 07:00:00
3,0,0,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,147,2025-02-25 12:00:00
4,3,1,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,217,2025-03-04 08:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,251,2025-03-01 21:00:00
247,2,7,2,8,1,1,0,3,0,0,...,2,2,1,13,6,2,1,1,97,2025-02-08 03:00:00
248,7,0,6,1,1,2,3,0,0,0,...,0,1,0,0,0,4,2,0,12,2025-02-23 08:00:00
249,21,8,1,10,12,14,5,10,0,3,...,0,0,2,4,5,4,0,14,45,2025-02-21 15:00:00


In [10]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-05 02:10:33,406 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 02:10:33,763 INFO: Initializing external client
2025-03-05 02:10:33,763 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 02:10:34,428 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215683
Fetching data from 2025-02-04 07:10:33.406455+00:00 to 2025-03-05 06:10:33.406455+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.97s) 
Skipping location_id 176: Not enough data to create even one window.


In [11]:
current_date

Timestamp('2025-03-05 07:10:33.406455+0000', tz='Etc/UTC')

In [12]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-04 08:00:00
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,2025-03-04 08:00:00
2,11,7,3,2,3,2,1,7,5,3,...,4,4,3,0,0,1,5,16,4,2025-03-04 08:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,2025-03-04 08:00:00
4,1,2,3,2,3,1,2,4,1,0,...,0,1,0,0,0,0,5,7,7,2025-03-04 08:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,2,259,2025-03-04 08:00:00
247,2,0,1,1,1,0,1,1,0,0,...,0,1,0,0,0,1,1,2,260,2025-03-04 08:00:00
248,5,16,21,34,25,31,37,28,33,48,...,1,0,0,1,1,1,1,9,261,2025-03-04 08:00:00
249,177,156,102,81,88,81,79,97,95,105,...,2,2,1,4,4,16,87,155,262,2025-03-04 08:00:00


In [13]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-05 02:13:18,859 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 02:13:18,875 INFO: Initializing external client
2025-03-05 02:13:18,877 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 02:13:19,601 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215683


In [14]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  File "d:\Taxi_Project\nyc_taxi\taxi_venv\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\subprocess.py", line 1538, in _execute_child
   

In [15]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,9.0
3,6,0.0
4,7,4.0
...,...,...
246,259,0.0
247,260,1.0
248,261,8.0
249,262,169.0


In [16]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([236, 186, 237, 239, 162, 142, 140, 170, 262, 238])